# KM003C PD Protocol Analysis - Complete Understanding

Based on official KM003C documentation and extensive analysis, we now have complete understanding of the protocol structure.

## Key Discoveries

### ADC Packet Structure (All Variants)
```c
// Main header (4 bytes) - Always present
struct data {
    uint32_t type : 7;    // 65 (CMD_PUT_DATA)  
    uint32_t extend : 1;  // Purpose unknown, not size indicator
    uint32_t id : 8;      // Transaction ID
    uint32_t : 6;         // Unused  
    uint32_t obj : 10;    // Object count (total packet size / 4)
}

// Extended header (4 bytes) - Always present for ADC
struct header {
    uint32_t att : 15;    // 1 (ATT_ADC)
    uint32_t next : 1;    // 0=standard, 1=has PD extension  
    uint32_t chunk : 6;   // Always 0 for ADC
    uint32_t size : 10;   // ADC payload size (40-44 bytes)
}

// ADC Data (48 bytes) - Complete measurement structure
typedef struct {
    int32_t Vbus;         // 1µV units
    int32_t Ibus;         // 1µA units  
    int32_t Vbus_avg;     // Smoothed voltage
    int32_t Ibus_avg;     // Smoothed current
    int32_t Vbus_ori_avg; // Uncalibrated voltage
    int32_t Ibus_ori_avg; // Uncalibrated current
    int16_t Temp;         // Internal temperature
    uint16_t Vcc1;        // CC1 voltage (0.1mV)
    uint16_t Vcc2;        // CC2 voltage (0.1mV) 
    uint16_t Vdp;         // D+ voltage (0.1mV)
    uint16_t Vdm;         // D- voltage (0.1mV)
    uint16_t Vdd;         // Internal VDD voltage (0.1mV)
    uint8_t Rate:2;       // Sample rate
    uint8_t n[3];         // Reserved
} AdcData_TypeDef;

// Optional PD Extension (12-32 bytes)
// Present when next=1 in extended header
// Contains PD timestamps, events, additional measurements
```

### Packet Variants by Size
- **Standard ADC**: obj=0x202 → 52 bytes (4+4+44 bytes)
- **Extended PD v1**: obj=0x203 → 68 bytes (52 + 16 PD bytes)  
- **Extended PD v2**: obj=0x204 → 84 bytes (52 + 32 PD bytes)

### Command Types Discovered
- **CmdGetSimpleAdcData** → **SimpleAdcData** (standard ADC)
- **CmdGetSimpleAdcData** → **Extended ADC + PD data** (PD mode)
- **CmdGetPdData** → **PdRawData** (pure PD protocol capture)

## Investigation Focus

**PD Files Analysis:**
- `orig_with_pd.13`: 347 PD packets + 128 ADC + 19 extended ADC
- `pd_capture_new.9`: 310 PD packets + 1,401 ADC + 18 extended ADC

In [ ]:
import polars as pl
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# Setup paths
project_root = Path.cwd()
while not (project_root / 'pyproject.toml').exists():
    project_root = project_root.parent
    
analysis_scripts_path = project_root / 'analysis' / 'scripts'
sys.path.insert(0, str(analysis_scripts_path))

import helpers
from km003c_lib import parse_packet

# Load dataset
df = helpers.load_master_dataset(project_root / 'usb_master_dataset.parquet')

print("✅ Dataset loaded successfully!")
print(f"Total packets: {len(df)}")

# Analyze both PD files with complete understanding
pd_files = ['orig_with_pd.13', 'pd_capture_new.9']

for pd_file in pd_files:
    print(f"\n{'='*50}")
    print(f"📋 ANALYZING: {pd_file}")
    print('='*50)
    
    # Get transactions for this file
    pd_df = df.filter(pl.col('source_file') == pd_file)
    transactions = helpers.get_transactions(pd_df, filter_out_enumeration=True)
    
    # Analyze packet type distribution
    packet_dist = transactions.group_by(['type', 'packet_type']).agg(
        pl.len().alias('count')
    ).sort('count', descending=True)
    
    print("Packet type distribution:")
    print(packet_dist)
    
    # Focus on ADC packets with different sizes
    adc_responses = transactions.filter(
        (pl.col('type') == 'Device Response') & 
        (pl.col('payload_hex').str.starts_with('41'))
    )
    
    if len(adc_responses) > 0:
        size_dist = adc_responses.group_by('payload_length').agg([
            pl.len().alias('count'),
            pl.col('packet_type').first().alias('parsed_type')
        ]).sort('payload_length')
        
        print(f"\nADC response size analysis:")
        for row in size_dist.iter_rows(named=True):
            size = row['payload_length'] 
            count = row['count']
            parsed = row['parsed_type'] or 'UNPARSED'
            
            # Decode what this size means based on our understanding
            if size == 52:
                analysis = "Standard ADC (4+4+44 bytes)"
            elif size == 68:
                analysis = "Extended ADC v1 (52+16 PD bytes)" 
            elif size == 84:
                analysis = "Extended ADC v2 (52+32 PD bytes)"
            else:
                analysis = f"Unknown variant ({size} bytes)"
                
            print(f"  {size:2d} bytes: {count:3d} packets [{parsed:15s}] → {analysis}")

def analyze_extended_adc_structure(pd_file):
    """Analyze the structure of extended ADC packets in detail."""
    print(f"\n🔬 EXTENDED ADC STRUCTURE ANALYSIS: {pd_file}")
    print("-" * 60)
    
    pd_df = df.filter(pl.col('source_file') == pd_file)
    transactions = helpers.get_transactions(pd_df, filter_out_enumeration=True)
    
    # Get extended ADC packets (68+ bytes)
    extended_adc = transactions.filter(
        (pl.col('payload_hex').str.starts_with('41')) &
        (pl.col('payload_length') >= 68)
    ).head(3)
    
    for row in extended_adc.iter_rows(named=True):
        payload = row['payload_hex']
        size = row['payload_length']
        payload_bytes = bytes.fromhex(payload)
        
        print(f"\nExtended ADC packet ({size} bytes):")
        print(f"  Time: {row['start_time']:.3f}s")
        
        # Main header (bytes 0-3)
        main_header = payload_bytes[:4]
        main_int = int.from_bytes(main_header, 'little')
        
        obj_value = (main_int >> 16) & 0x3FF
        extend_bit = (main_int >> 7) & 0x1
        
        print(f"  Main header: {' '.join(f'{b:02x}' for b in main_header)}")
        print(f"    obj=0x{obj_value:03x} ({obj_value}), extend={extend_bit}")
        
        # Extended header (bytes 4-7) 
        ext_header = payload_bytes[4:8]
        ext_int = int.from_bytes(ext_header, 'little')
        
        att = ext_int & 0x7FFF
        next_bit = (ext_int >> 15) & 0x1
        size_field = (ext_int >> 22) & 0x3FF
        
        print(f"  Ext header: {' '.join(f'{b:02x}' for b in ext_header)}")
        print(f"    att={att} (ATT_ADC), next={next_bit}, size={size_field}")
        
        # ADC data (bytes 8-55, should be 48 bytes)
        adc_data = payload_bytes[8:56]
        print(f"  ADC data: {len(adc_data)} bytes")
        
        # Try to parse first 52 bytes (main+ext header + standard ADC)
        try:
            standard_packet = payload_bytes[:52]
            parsed = parse_packet(standard_packet)
            if parsed.adc_data:
                adc = parsed.adc_data
                print(f"    ✅ Standard ADC: V={adc.vbus_v:.3f}V I={adc.ibus_a:.3f}A T={adc.temp_c:.1f}°C")
        except Exception as e:
            print(f"    ❌ Parse error: {e}")
            
        # Extension data
        if size > 56:
            extension = payload_bytes[56:]
            print(f"  PD extension: {len(extension)} bytes")
            ext_hex = ' '.join(f'{b:02x}' for b in extension[:16])
            if len(extension) > 16:
                ext_hex += "..."
            print(f"    Data: {ext_hex}")

# Analyze extended ADC structure for both files
for pd_file in pd_files:
    analyze_extended_adc_structure(pd_file)

In [4]:
# Step 1: Examine the two PD files

pd_files = ['orig_with_pd.13', 'pd_capture_new.9']

for pd_file in pd_files:
    print(f"\n{'='*60}")
    print(f"FILE: {pd_file}")
    print('='*60)
    
    # Filter for this PD file
    pd_df = df.filter(pl.col('source_file') == pd_file)
    
    print(f"Packets in {pd_file}: {len(pd_df)}")
    print(f"Time range: {pd_df['timestamp'].min():.3f}s - {pd_df['timestamp'].max():.3f}s")
    print(f"Duration: {pd_df['timestamp'].max() - pd_df['timestamp'].min():.3f}s")
    
    # Show packet overview
    packet_overview = pd_df.group_by(['transfer_type', 'direction']).agg([
        pl.len().alias('count'),
        pl.col('data_length').mean().alias('avg_length')
    ]).sort('count', descending=True)
    
    print(f"\nPacket types in {pd_file}:")
    print(packet_overview)


FILE: orig_with_pd.13
Packets in orig_with_pd.13: 2056
Time range: 0.000s - 42.093s
Duration: 42.093s

Packet types in orig_with_pd.13:
shape: (3, 4)
┌───────────────┬───────────┬───────┬────────────┐
│ transfer_type ┆ direction ┆ count ┆ avg_length │
│ ---           ┆ ---       ┆ ---   ┆ ---        │
│ str           ┆ str       ┆ u32   ┆ f64        │
╞═══════════════╪═══════════╪═══════╪════════════╡
│ 0x03          ┆ D->H      ┆ 1022  ┆ 15.225049  │
│ 0x03          ┆ H->D      ┆ 1008  ┆ 2.15873    │
│ 0x02          ┆ D->H      ┆ 26    ┆ 13.038462  │
└───────────────┴───────────┴───────┴────────────┘

FILE: pd_capture_new.9
Packets in pd_capture_new.9: 6930
Time range: 0.000s - 295.600s
Duration: 295.600s

Packet types in pd_capture_new.9:
shape: (3, 4)
┌───────────────┬───────────┬───────┬────────────┐
│ transfer_type ┆ direction ┆ count ┆ avg_length │
│ ---           ┆ ---       ┆ ---   ┆ ---        │
│ str           ┆ str       ┆ u32   ┆ f64        │
╞═══════════════╪═══════════╪═

## PD Protocol Investigation Notes

Use this section to document findings about the KM003C PD protocol functionality:

### Key Observations
- [ ] PD packet identification patterns
- [ ] Command/response structures  
- [ ] Timing characteristics
- [ ] Error handling patterns

### Protocol Structure
- [ ] PD message types discovered
- [ ] Payload format analysis
- [ ] Transaction flow patterns

### Next Steps
- [ ] Implement PD-specific packet parsing
- [ ] Add PD message decoding
- [ ] Create PD visualization tools